In [13]:
# Inladen van data
# Verkennen van data / visualiseren
# Web applicatie
# 

INFILE  = "/Users/bramzijlstra/git/kvk/pocs/012_hackwerk/files/Ondernemingen.json"
OUTFILE = "/Users/bramzijlstra/git/kvk/pocs/012_hackwerk/output/test.json"

total = 0
import ijson
from pathlib import Path    
from pprint import pprint
Path(OUTFILE).parent.mkdir(parents=True, exist_ok=True)  

data = []

with open(INFILE, "r", encoding="utf-8") as fin, open(OUTFILE, "w", encoding="utf-8") as fout:     
     # We schrijven een geldige JSON-array, streamend.                                            
     fout.write("[")                                                                              
     first_out = True                                                                              
                                                                                                  
     iterator = ijson.items(fin, "item")                                                           
                                                                                                  
     for obj in iterator:
         plaats = obj["Plaats_RechtsPersoon"]
         if plaats in ['Amsterdam', 'Rotterdam', 'Utrecht', 'Eindhoven'] and obj['BedrijfsOmschrijving']:
             data.append(obj)
             
             total += 1

         else:
             continue
    
         if total > 99:
             break

In [25]:
for i in data:
    print(i['BedrijfsOmschrijving'])

Het leveren van verantwoorde medisch specialistische zorg op het gebied van orthopedie; het verlenen van medische en humanitaire hulp aan Suriname in samenwerking met daarvoor in aanmerking komende instellingen en organisaties
Tegen vergoeding aanbieden van netwerkplekken tijdens een netwerkevenement
Het uitoefenen van een onderneming, welke zich bezig houdt:
- met medische specialistische zorgverlening;
- aanvullende medische en paramedische zorgverlening.
Het realiseren van diensten in zorg en welzijn.
Het bevorderen van de mogelijkheden tot sportbeoefening;
het creëren van werkgelegenheid in de sport, waaronder
de mogelijkheden tot stages
Buitenschoolse opvang op sport gericht
Het voorlichten en adviseren van woningeigenaren en/of VvE's die energiebesparende maatregelen willen nemen
Advies en automatisering, zijnde het ontwikkelen (programmeren, ontwerpen van programmatuur, alsmede het beheren van deze systemen. In het bijzonder daar waar burgerrechten, mensenrechten en beveiliging 

In [16]:
import streamlit as st
import pandas as pd
from geopy.geocoders import Nominatim
import time

In [17]:
df = pd.DataFrame(data)

geolocator = Nominatim(user_agent="plaats-mapper")

# Coördinaten ophalen
latitudes, longitudes = [], []
for plaats in df["Plaats_RechtsPersoon"]:
    loc = geolocator.geocode(f"{plaats}, Nederland")
    if loc:
        latitudes.append(loc.latitude)
        longitudes.append(loc.longitude)
    else:
        latitudes.append(None)
        longitudes.append(None)
    time.sleep(1)  # om blokkeren te voorkomen

df["lat"] = latitudes
df["lon"] = longitudes

# Plotten in Streamlit
st.map(df[["lat", "lon"]])

2025-09-19 08:11:16.805 WARNING streamlit.runtime.scriptrunner_utils.script_run_context: Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-19 08:11:16.890 
  command:

    streamlit run /Users/bramzijlstra/.pyenv/versions/3.10.10/envs/kvk_main/lib/python3.10/site-packages/ipykernel_launcher.py [ARGUMENTS]
2025-09-19 08:11:16.890 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [21]:
df = pd.DataFrame(data) 
import numpy as np
# Bounding box ranges per stad (lat_min, lat_max, lon_min, lon_max)
ranges = {
    "Amsterdam": (52.28, 52.43, 4.73, 5.02),
    "Utrecht":   (52.05, 52.14, 5.04, 5.18),
    "Rotterdam": (51.85, 51.98, 4.38, 4.61),
    "Eindhoven": (51.38, 51.51, 5.38, 5.53),
}

# Functie om random lat/lon te genereren
def random_coords(city):
    if city not in ranges:
        return (np.nan, np.nan)
    lat_min, lat_max, lon_min, lon_max = ranges[city]
    lat = np.random.uniform(lat_min, lat_max)
    lon = np.random.uniform(lon_min, lon_max)
    return lat, lon

# Toepassen op de DataFrame
df[["lat", "lon"]] = df["Plaats_RechtsPersoon"].apply(lambda c: pd.Series(random_coords(c)))


In [23]:
df.to_csv("kvk_data_codam.csv", index=False)

In [43]:
# !pip install -U streamlit streamlit-jupyter geopy pandas pydeck

# 1) Patch Streamlit to render inside Jupyter (must run before importing streamlit)
from streamlit_jupyter import StreamlitPatcher
StreamlitPatcher().jupyter()

# 2) Imports
import streamlit as st
import pandas as pd
import pydeck as pdk

# --- Your data here ---
# If you already have lat/lon columns, just build df with those and skip geocoding.

df = pd.DataFrame(data)

# --- OPTIONAL: Geocode if lat/lon missing ---
if not {"lat","lon"}.issubset(df.columns):
    from geopy.geocoders import Nominatim
    import time
    from functools import lru_cache

    geolocator = Nominatim(user_agent="nl-places-mapper")

    @lru_cache(maxsize=None)
    def geocode_place(plaats):
        loc = geolocator.geocode(f"{plaats}, Nederland")
        return (loc.latitude, loc.longitude) if loc else (None, None)

    lats, lons = [], []
    for plaats in df["Plaats_RechtsPersoon"]:
        lat, lon = geocode_place(str(plaats))
        lats.append(lat); lons.append(lon)
        time.sleep(1)  # be polite to the free OSM service
    df["lat"] = lats
    df["lon"] = lons

# Clean rows without coords
points = df.dropna(subset=["lat","lon"]).copy()

# 3) Center the view on the Netherlands and plot each row
st.title("Kaart van Nederland – punten per rij")
view_state = pdk.ViewState(latitude=52.1, longitude=5.3, zoom=6)

layer = pdk.Layer(
    "ScatterplotLayer",
    data=points,
    get_position='[lon, lat]',
    get_radius=6000,        # ~6 km radius marker
    pickable=True,
)

tooltip = {"text": "{Plaats_RechtsPersoon}\n({lat}, {lon})"}

st.pydeck_chart(
    pdk.Deck(
        layers=[layer],
        initial_view_state=view_state,
        tooltip=tooltip,
        map_style="light",
    )
)

# Kaart van Nederland – punten per rij

2025-09-18 17:01:10.276 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 17:01:10.279 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.
2025-09-18 17:01:10.279 Thread 'MainThread': missing ScriptRunContext! This warning can be ignored when running in bare mode.


DeltaGenerator()

In [47]:
# pip install plotly pandas
import pandas as pd
import plotly.express as px


fig = px.scatter_map(
    df, lat="lat", lon="lon",
    hover_name="Plaats_RechtsPersoon",
    zoom=6, center={"lat": 52.1, "lon": 5.3},
    map_style="open-street-map"   # andere opties: "carto-positron", "stamen-terrain", etc.
)
fig.update_layout(margin=dict(l=0, r=0, t=0, b=0))
fig.show()

In [24]:
df

,Id,KVKNummer,VestigingNummer,LaatstGewijzigdTijdstip_MA,HoofdNevenVestiging,Actief,Indicator_Onderneming,Indicator_Vestiging,ImportIndicator,ExportIndicator,...,Gemeente_RechtsPersoon,CoropCode_RechtsPersoon,Corop_RechtsPersoon,Provincie_RechtsPersoon,Regio_RechtsPersoon,KrimpGebied_RechtsPersoon,AnticipeerGebied_RechtsPersoon,IbisPlanNaam_RechtsPersoon,lat,lon
0,1178628,015828107,09867709295,2020-04-28 13:36:07.8710000,Hoofdvestiging,Ja,Ja,Ja,Nee,Nee,...,Amsterdam,CR23,Groot-Amsterdam,Noord-Holland,Noordwest,Nee,Nee,zz - n.v.t./onbekend,52.309344,4.884548
1,13598565,006451423,03772422261,2022-06-28 13:44:49.0010000,Hoofdvestiging,Nee,Ja,Ja,Nee,Nee,...,Eindhoven,CR36,Zuidoost-Noord-Brabant,Noord-Brabant,Zuid,Nee,Nee,zz - n.v.t./onbekend,51.450731,5.500133
2,1973929,051804166,03835681349,2022-05-25 12:16:08.2820000,Hoofdvestiging,Ja,Ja,Ja,Nee,Nee,...,Eindhoven,CR36,Zuidoost-Noord-Brabant,Noord-Brabant,Zuid,Nee,Nee,zz - n.v.t./onbekend,51.450962,5.453899
3,11735140,063572655,07265274934,2018-01-09 14:28:02.7950000,Hoofdvestiging,Nee,Ja,Ja,Nee,Nee,...,Rotterdam,CR29,Groot-Rijnmond,Zuid-Holland,Zuidwest,Nee,Nee,zz - n.v.t./onbekend,51.920720,4.433812
4,13583448,040644122,07089340237,2019-08-12 11:18:54.0690000,Hoofdvestiging,Nee,Ja,Ja,Nee,Nee,...,Amsterdam,CR23,Groot-Amsterdam,Noord-Holland,Noordwest,Nee,Nee,zz - n.v.t./onbekend,52.318757,4.919727
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
95,5142755,010224535,03201183102,2014-04-12 10:38:36.0710000,Hoofdvestiging,Ja,Ja,Ja,Nee,Nee,...,Rotterdam,CR29,Groot-Rijnmond,Zuid-Holland,Zuidwest,Nee,Nee,zz - n.v.t./onbekend,51.917944,4.507244
96,12864416,040222651,00684178136,2013-12-24 14:51:01.9150000,Hoofdvestiging,Nee,Ja,Ja,Nee,Nee,...,Rotterdam,CR29,Groot-Rijnmond,Zuid-Holland,Zuidwest,Nee,Nee,Spaanse Polder,51.929273,4.382260
97,6506232,059143663,08046332552,2022-05-19 10:07:27.1120000,Hoofdvestiging,Ja,Ja,Ja,Nee,Nee,...,Rotterdam,CR29,Groot-Rijnmond,Zuid-Holland,Zuidwest,Nee,Nee,zz - n.v.t./onbekend,51.954197,4.517555
98,12063778,098257259,05704011800,2021-09-22 09:06:48.8030000,Hoofdvestiging,Nee,Ja,Ja,Nee,Nee,...,Rotterdam,CR29,Groot-Rijnmond,Zuid-Holland,Zuidwest,Nee,Nee,zz - n.v.t./onbekend,51.916851,4.565595
